### **1. Importing libraries**

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from utils.utils import split_data
from crack_segmentation.subroutines.loss_metrics import (
    Weighted_Cross_Entropy,
    F1_score_dil,
    F1_score,
    Precision_dil
)

### **2. Loading the dataset**

In [2]:
dataset = pd.read_csv('../dataset/dataset_final.csv')

dataset['Filepath'] = dataset['Filepath'].apply(lambda x: '../' + x)

train_df, test_df = train_test_split(
    dataset.sample(6000, random_state=42),
    train_size=0.80,
    random_state=42
)

preprocess_input = tf.keras.applications.resnet.preprocess_input

train_data, valid_data, test_data = split_data(
        train_df, 
        test_df, 
        image_height=224,
        image_width=224,
        image_channels=1.0, 
        preprocess_input=preprocess_input,
    )

Found 3840 validated image filenames belonging to 2 classes.
Found 960 validated image filenames belonging to 2 classes.
Found 1200 validated image filenames belonging to 2 classes.


### **3. Importing the UNet-ResNet50 Model**

In [3]:
model = load_model(
    'crack_segmentation/output/checkpoints/unet_resnet50.h5',
    custom_objects={
        'loss': Weighted_Cross_Entropy(10),
        'F1_score': F1_score,
        'F1_score_dil': F1_score_dil,
        'Precision_dil': Precision_dil
    },
    compile=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

2024-03-12 17:14:20.663482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-12 17:14:20.693761: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cilab_user/surface_crack_detection/cilab/lib/python3.10/site-packages/cv2/../../lib64:
2024-03-12 17:14:20.693783: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-03-12 17:14:20.694511: I tensorflow/core/platfor

### **4. Evaluating the model**

In [4]:
loss, acc = model.evaluate(test_data)

print(f"loss' model: {loss}")
print(f"accuracy's model: {acc}")


38/38 [==============================] - 144s 4s/step - loss: 2.9397 - accuracy: 0.9533
loss' model: 2.93967604637146
accuracy's model: 0.95333331823349


### **5. Making predictions**

In [5]:
y_pred = np.argmax(model.predict(test_data), axis=-1)
metrics = classification_report(test_data.labels, y_pred)

print(metrics)

38/38 [==============================] - 142s 4s/step
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       576
           1       0.99      0.92      0.95       624

    accuracy                           0.95      1200
   macro avg       0.95      0.95      0.95      1200
weighted avg       0.96      0.95      0.95      1200

